In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import nltk
from  nltk.sentiment import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import wikipediaapi as wikiapi
import imdb as imdb
import requests
from SPARQLWrapper import SPARQLWrapper
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/youssefamine/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [68]:
wiki = wikiapi.Wikipedia('MyProjectName (merlin@example.com)')

In [46]:
PATH = 'cmu/'
movies = pd.read_csv(PATH + 'movie.metadata.tsv', sep='\t', header=None)
summaries = pd.read_csv(PATH + 'plot_summaries.txt', sep='\t', header=None)
summaries
summary_columns = ['Wikipedia_Movie_ID', 'Plot_Summary']
summaries.columns = summary_columns
summaries

,Wikipedia_Movie_ID,Plot_Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


In [3]:
plot_summaries = summaries['Plot_Summary']
summary = str(plot_summaries[0])
print(summary)

Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.


In [117]:
sid = SentimentIntensityAnalyzer()
print(summary)
ss = sid.polarity_scores(summary)
print(ss)
for k in sorted(ss):
   print('{0}: {1}, '.format(k, ss[k]), end='')


1940 - Operation Dynamo has just taken place. From the newly conquered French coastline, a Wehrmacht colonel looks out over the English Channel with powerful binoculars. Surveying the white cliffs of Dover, he spies Godfrey emerging from a lavatory. Godfrey joins the rest of his platoon, who are defiantly waving the Union Flag. The colonel fumes contemptuously, "How can the stupid British ever hope to win?!" One morning, George Mainwaring, the manager of the Walmington-on-Sea branch of Martins Bank, and his chief clerk, Arthur Wilson, listen to Anthony Eden making a radio broadcast about forming the Local Defence Volunteers.The actual talk was given in the evening, when most people would be listening to their radios. At the local police station chaos ensues because there is nobody to organise the enrolment of the men. Characteristically, Mainwaring takes charge and after commandeering the local church hall he registers the assembled volunteers. The local platoon is eventually formed wi

In [47]:
data = {'Wikipedia_Movie_ID': [], 'compound': [], 'negative': [], 'neutral': [], 'postive': []}
for i in range(len(plot_summaries)):
    row = summaries.iloc[i]
    summary = str(row['Plot_Summary'])
    ids = row['Wikipedia_Movie_ID']
    ss = sid.polarity_scores(summary)
    data['Wikipedia_Movie_ID'].append(ids)
    data['compound'].append(ss['compound'])
    data['negative'].append(ss['neg'])
    data['neutral'].append(ss['neu'])
    data['postive'].append(ss['pos'])

In [50]:
emotions = pd.DataFrame(data).sort_values(by='Wikipedia_Movie_ID')
emotions

,Wikipedia_Movie_ID,compound,negative,neutral,postive
2100,330,0.2023,0.000,0.938,0.062
6038,3217,-0.9831,0.138,0.782,0.081
20555,3333,0.5921,0.097,0.809,0.095
39373,3746,-0.9798,0.116,0.805,0.079
13609,3837,-0.9678,0.105,0.831,0.064
...,...,...,...,...,...
20723,37373877,-0.7269,0.126,0.764,0.110
26062,37473592,-0.2500,0.045,0.922,0.033
33634,37478048,0.9837,0.061,0.822,0.117
16838,37492363,0.9643,0.053,0.604,0.343


In [119]:
character_columns = ['Wikipedia_Movie_ID', 'Freebase_Movie_ID', 'Character Name', 'Actor DOB', 'Actor gender', 'Actor height', 'Actor ethnicity', 'Actor Name', 'Actor age at movie release', 'Freebase character map']
movies_columns = ['Wikipedia_Movie_ID', 'Freebase_Movie_ID', 'Movie_name', 'Movie_release_date', 'Movie_box_office_revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']
movies.columns = movies_columns
print(movies_columns)

['Wikipedia_Movie_ID', 'Freebase_Movie_ID', 'Movie_name', 'Movie_release_date', 'Movie_box_office_revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']


In [125]:
box_movies = movies[['Wikipedia_Movie_ID', 'Movie_name', 'Movie_box_office_revenue', 'Movie_release_date', 'Freebase_Movie_ID']].sort_values('Wikipedia_Movie_ID', ascending=True)
box_emotions = emotions.join(box_movies, on='Wikipedia_Movie_ID', lsuffix="_l", rsuffix='_r').drop(['Wikipedia_Movie_ID_r', 'Wikipedia_Movie_ID_l'], axis=1)
box_emotions
box_movies

,Wikipedia_Movie_ID,Movie_name,Movie_box_office_revenue,Movie_release_date,Freebase_Movie_ID
61305,330,Actrius,NaN,1996,/m/0ktn59
14570,3217,Army of Darkness,21502796.0,1992-10-09,/m/014hr
2634,3333,The Birth of a Nation,50000000.0,1915,/m/0151l
48191,3746,Blade Runner,33139618.0,1982-06-25,/m/017n9
48856,3837,Blazing Saddles,119500000.0,1974-02-07,/m/018f8
...,...,...,...,...,...
2674,37473592,Thoppul Kodi,NaN,2011-12-16,/m/0h67507
24619,37476824,I Love New Year,NaN,2011,/m/09ck55
81283,37478048,Mr. Bechara,NaN,1996,/m/0g57c0y
73787,37492363,Cherries and Clover,NaN,2011,/m/0ds7zbt


In [124]:
def get_wikidata_id(freebase_id):
    # Define SPARQL endpoint and query
    url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P646 "{freebase_id}".
    }}
    """
    headers = {"Accept": "application/sparql-results+json"}
    # Execute the query
    response = requests.get(url, params={'query': query}, headers=headers)
    data = response.json()
    return data


get_wikidata_id('/m/014hr')


{'head': {'vars': ['item']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q471992'}}]}}

In [115]:
api_key = 'f8771939d394040dc6b642d4dc497043'
tmdb_id = 'Actrius'
url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={api_key}"
response = requests.get(url)
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code}")

Error: 404
